### Imports

In [2]:
import pandas as pd
import numpy as np

#### Source

http://robotics.ethz.ch/~asl-datasets/ijrr_euroc_mav_dataset/machine_hall/


#### Background

The flight environment (an indoor factory hall)
Easy → the level of difficulty of the flight path

#### Data

IMU_data/data.csv → IMU data (what the drone measured)

This is Analogous to kite’s onboard sensors.

Sampling rate = 200 Hz -> dt =  0.005 s.



state_groundtruth_data/data.csv (Ground-truth data) measured with A motion-capture system providing ground-truth position

-> contains (v_x, v_y, v_z) linear velocity (m/s) which is target

Sampling rate = 100 Hz -> dt =  0.01 s.

#### Task description

Will use RNN to predict velocity (vx,vy,vz) from IMU data.

#### Data handling

Currently, the IMU measurements and the groundtruth have different timestamps dt = 0.005 for IMU and dt = 0.01 for groundtruth -> Resample both to dt = 0.02.

In [ ]:
import torch

IMU = pd.read_csv("../Data/IMU_data/data.csv")
groundtruth = pd.read_csv("../Data/state_groundtruth_data/data.csv")

IMU.drop(index = 0, inplace = True) #Because of different timestamps

"""Adjust such that timestamp start at 0 and so that time-spaces are even in Input and output"""

IMU['t'] = (IMU['#timestamp [ns]'] - IMU['#timestamp [ns]'].iloc[0]) * 1e-9
groundtruth['t'] = (groundtruth['#timestamp'] - groundtruth['#timestamp'].iloc[0]) * 1e-9

IMU = IMU.iloc[:len(groundtruth)]


""""""
IMU.drop(columns=['#timestamp [ns]'], inplace=True)
IMU.drop(columns=['t'], inplace=True, errors='ignore')  # in case you created t

groundtruth.drop(columns=['#timestamp'], inplace=True)
groundtruth.drop(columns = ['t'], inplace =True)

X = IMU.to_numpy()
Y = groundtruth.to_numpy()

print(X.shape)
print(X)

X = torch.tensor(IMU, dtype=torch.float32)



(36382, 6)
[[-0.0991347   0.14032447  0.02932153  8.03328079 -0.40861042 -2.40262925]
 [-0.09843657  0.1277581   0.03769911  7.88618104 -0.42495483 -2.43531808]
 [-0.10262536  0.11588986  0.04537856  7.82897558 -0.37592158 -2.46800692]
 ...
 [ 0.04258603  0.01186824  0.31136674  8.59716317  0.42495483 -3.19533346]
 [ 0.06003933  0.1563815   0.32114058 10.36236017 -0.21247742 -3.45684412]
 [ 0.05654867  0.04886922  0.3260275   9.65955025 -0.03268883 -3.138128  ]]
